Connect to database

In [3]:
import datajoint as dj

dj.config['database.host'] = "gl-ash.biostr.washington.edu"
dj.config['database.user'] = "gabby"
dj.config['database.port'] = 3306

dj.conn()

[2026-02-18 13:38:31,013][INFO]: DataJoint 0.14.6 connected to gabby@gl-ash.biostr.washington.edu:3306


DataJoint connection (connected) gabby@gl-ash.biostr.washington.edu:3306

Imports

In [11]:
import os
import numpy as np

import spyglass.common as sgc
import spyglass.spikesorting.v1 as sgs

from gl_spyglass.utils.common_neural_functions import validate_references

In [5]:
os.environ['FIGPACK_BUCKET'] = 'gillespielab'

# TODO: update this with your own custom FIGPACK_API_KEY (contact Jeremy Magland if you don't have one)
os.environ['FIGPACK_API_KEY'] = '17979e3bc8880ea91520c1dc2e8ab6a97fc665893fe2728e9d642923c3dcf31c'

Choose tetrodes for your relevant subj(s), date(s)

In [7]:
subj_sess_tetrodes = {
    'pippin': {
        20210421: {
            'can1_ca1_tetrodes': [6, 9, 30],
            'can2_ca1_tetrodes': [35, 36, 40],
        },
    },
}

Choose parameters

Choose between `plot_type` 'one_from_many_tet' or 'all_from_one_tet' depending on which kind of artifact detection you're visualizing. For coincicent spike detection, it's useful to look at one electrode from many tetrodes so that you can see if the cross-tetrode artifacts are getting caught. For threshold artifact detection (from `ArtifactDetection`) it's useful to look at all the electrodes from a single tetrode because it works per sort group (tetrode) to look for big artifacts that are present across multiple electrodes in the tetrode).

In [8]:
subj = 'pippin'
date = 20210421
nwb_file_name = f'{subj}{date}_.nwb'
interval_list_name = '02_r1'
plot_type = 'one_from_many_tet' 

In [12]:
sgs.SortGroup.SortGroupElectrode()

nwb_file_name name of the NWB file,sort_group_id,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode
archibald20210711_.nwb,0,0,0
archibald20210711_.nwb,0,0,1
archibald20210711_.nwb,0,0,2
archibald20210711_.nwb,0,0,3
archibald20210711_.nwb,1,1,4
archibald20210711_.nwb,1,1,5
archibald20210711_.nwb,1,1,6
archibald20210711_.nwb,1,1,7
archibald20210711_.nwb,2,2,8
archibald20210711_.nwb,2,2,9


In [ ]:
subj_info = subj_sess_tetrodes[subj][date]
epoch = int(interval_list_name[:2])
pos_interval_list_name = (sgc.IntervalList() & {'nwb_file_name': nwb_file_name, 'pipeline': 'position'}).fetch('interval_list_name')[epoch - 1]
sort_interval_list_name = interval_list_name

if plot_type == 'one_from_many_tet':
    can1_ca1_tetrodes = subj_info['can1_ca1_tetrodes']
    can2_ca1_tetrodes = subj_info['can2_ca1_tetrodes']

    # get electrodes info
    electrodes_df, val_can_refs = validate_references(nwb_file_name, is_copy=True)

    # narrow down electrodes_df to good electrodes
    electrodes_df = electrodes_df[electrodes_df['bad_channel'] == 'False']

    # convert tetrode numbers to electrode group numbers 
    can1_ca1_elec_groups = np.asarray(can1_ca1_tetrodes) - 1
    can2_ca1_elec_groups = np.asarray(can2_ca1_tetrodes) - 1

    # find the electrode id for the first good channel on the tetrode
    can1_ca1_elecs = [electrodes_df.loc[electrodes_df['electrode_group_name'] == elec_group, 'electrode_id'].values[0] for elec_group in can1_ca1_elec_groups]
    can2_ca1_elecs = [electrodes_df.loc[electrodes_df['electrode_group_name'] == elec_group, 'electrode_id'].values[0] for elec_group in can2_ca1_elec_groups]

    # select elecs and elec labels
    plot_elecs = np.concatenate([can1_ca1_elecs, can2_ca1_elecs])
    plot_elec_labels = np.concatenate([[f'can1 {e + 1}: elec {elec}' for e, elec in enumerate(can1_ca1_elecs)], [f'can2 {e + 1}: elec {elec}' for e, elec in enumerate(can2_ca1_elecs)]])

if plot_type == 'all_from_one_tet':
    # select which cannula and tetrode index to plot from
    can = 1
    tet_idx = 0
    tetrode = subj_info[f'can{can}_ca1_tetrodes'][tet_idx]

    # get electrodes info
    electrodes_df, val_can_refs = validate_references(nwb_file_name, is_copy=True)

    # narrow down electrodes_df to good electrodes
    electrodes_df = electrodes_df[electrodes_df['bad_channel'] == 'False']

    # find which electrode ids corresponds to this tetrode
    elec_group = tetrode - 1
    plot_elecs = electrodes_df.loc[electrodes_df['electrode_group_name'] == elec_group, 'electrode_id'].values
    plot_elec_labels = np.asarray([f'can{can} {e + 1}: elec {elec}' for e, elec in enumerate(plot_elecs)])

# set colors
colors = ['grey']*len(plot_elecs)